In [2]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json

In [46]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

datetime.datetime(2026, 1, 9, 14, 43, 49, 28686)

In [4]:
project = hopsworks.login()
fs = project.get_feature_store() 

2026-01-09 13:49:06,252 INFO: Initializing external client
2026-01-09 13:49:06,254 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-09 13:49:07,968 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286325


In [5]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="flight_xgboost_model",
    version=1,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2026-01-09 13:49:14,883 INFO: Initializing for batch retrieval of feature vectors


Downloading: 0.000%|          | 0/447322 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/152799 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/17452 elapsed<00:00 remaining<?

In [6]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='2.779605E2', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['int', 'int', 'int', 'int', 'int'], gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## How the pipeline should look like 

Idea is to get todays data from google - save the prediction for tomorrows flights and then tomorrow fill that value with the correct prediction - save it to csv file: 

date --|-- predicted_landings --|-- total_landings 
10 jan -|--------- 185----------|------ NAN------

and then tomorrow when it is 10th of jan

date --|-- predicted_landings --|-- total_landings 
10 jan -|--------- 185----------|------ 200------
11 jan -|--------- 113----------|------ NAN------

In [59]:
# Retrieve the google trend values to predict tomorrows flights 
google_data_fg = fs.get_feature_group(
    name='google_trends_daily',
    version=1,
)

batch_data = google_data_fg.filter(google_data_fg.date == today).read()
X = batch_data.drop(columns=['date','city'])
y_pred = retrieved_xgboost_model.predict(X)
y_pred

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.03s) 


,vikings,fika,stockholm,ikea,abba


In [ ]:
forecast_row = pd.DataFrame({
    "date":[tomorrow],
    "city":["Märsta"],
    "predicted_landings":[y_pred],
    "total_landings":[None],
})

In [53]:
# Get or create feature group for storing the monitoring values 
monitor_fg = fs.get_or_create_feature_group(
    name='flight_prediction',
    description='Flight prediction monitoring',
    version=1,
    primary_key=['city']
    event_time= "date"
)

2026-01-09 15:24:51,506 ERROR: Parser Error: . Detail: Python exception: Traceback (most recent call last):
  File "/usr/src/app/src/server.py", line 142, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 166, in wrapper
    result = func(instance, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 196, in do_get
    return self._read_query(context, path, command)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 123, in wrapper
    return func(instance, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 131, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 227, in _read_query
    result_batches = self.hudi_query_engine.read_query(query_obj)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

FeatureStoreException: Could not read data using Hopsworks Query Service.

In [ ]:
monitor_fg.insert(forecast_row, wait=True)

In [48]:
# Retrieve feature group
flight_data_fg = fs.get_feature_group(
    name='flight_data_arlanda',
    version=1,
)

outcome_df = flight_data_fg.filter(flight_data_fg.date == today).read() # this give todays 

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
